# Assignment: Web Security

### Part 1: picoCTF

#### Exercise 1

#### Solution

After entering the web provided once the instace had started, we find a web page with multiple tabs. The instructions of the assignment hint us to inspect the different tabs of such webpage. After checking and inspecting the resulting page, we found out the following once we made use of the "Inspect" tool of the web browser:

<img src="src/1.png" width="700"/>

at first this discovery seems to be an encrypted message. By its length and used of letters and numbers one can guess that the encryption was made using a base64 algorithm. So, we went ahead and try to decode it:

In [2]:
import base64

#texto codificado
encoded_text = "cGljb0NURnt3ZWJfc3VjYzNzc2Z1bGx5X2QzYzBkZWRfZjZmNmI3OGF9"

# decodificación del texto
decoded_text = base64.b64decode(encoded_text).decode("utf-8")
print(decoded_text)


picoCTF{web_succ3ssfully_d3c0ded_f6f6b78a}


giving as a result the flag of interest.

#### Which OWASP Top 10 is Being Exploited?
The vulnerability being exploited in this scenario is Sensitive Data Exposure (OWASP A3: Sensitive Data Exposure) which occurs when an application fails to properly protect sensitive data, such as passwords, personal information, or cryptographic keys. In this scenario, we discovered encrypted data in the client-side code via the "Inspect" tool. This suggests that sensitive data was being exposed, even if encrypted. While the data might have been encrypted, it was still visible to us as users in the browser, potentially allowing anybody to decrypt it or gain insights into how to access sensitive information.

#### How Can It Be Patched?
To patch this vulnerability, it is essential to implement proper encryption mechanisms and ensure sensitive data is never exposed in the client-side code. Sensitive information should be encrypted at rest and in transit using strong algorithms (maybe AES as seen in class) and keys should never be exposed on the client side. Additionally, one should avoid storing sensitive data (even in encrypted form) in places accessible to the client, such as HTML or JavaScript code, and always use secure communication channels (TLS/SSL) for data transfer. It is also important to ensure that access control measures are enforced to prevent unauthorized access to sensitive resources.

#### Exersice 2

#### Solution
After entering the web provided once the instace had started, we find a login page. The instructions of the assignment hint us to inspect the login process of such webpage. After checking and inspecting the resulting page once we tried to logged in with a generic username and password, we found out the following once we made use of the "Inspect" tool of the web browser:

<img src="src/2.png" width="700"/>

```
filter(string) {
        filterPassed = true;
        for (let i =0; i < string.length; i++){
          cc = string.charCodeAt(i);
          
          if ( (cc >= 48 && cc <= 57) ||
               (cc >= 65 && cc <= 90) ||
               (cc >= 97 && cc <= 122) )
          {
            filterPassed = true;     
          }
          else
          {
            return false;
          }
        }
        
        return true;
      }
    
      window.username = "alex1234";
      window.password = "password1234";
      
      usernameFilterPassed = filter(window.username);
      passwordFilterPassed = filter(window.password);
      
      if ( usernameFilterPassed && passwordFilterPassed ) {
      
        loggedIn = checkPassword(window.username, window.password);
        
        if(loggedIn)
        {
          document.getElementById('msg').innerHTML = "Log In Successful";
          document.getElementById('adminFormHash').value = "2196812e91c29df34f5e217cfd639881";
          document.getElementById('hiddenAdminForm').submit();
        }
        else
        {
          document.getElementById('msg').innerHTML = "Log In Failed";
        }
      }
      else {
        document.getElementById('msg').innerHTML = "Illegal character in username or password."
      }
```
Given that the username (alex1234) and password (password1234) are hardcoded in the JavaScript, it’s likely that the checkPassword function simply compares these values to the user input. If the values match, it returns true, allowing the user to log in. So, after inspection a little bit more, still making use of the "inspect" tool of the web browser, we found the following code inside a file called secure.js: 

<img src="src/3.png" width="700"/>

```
function checkPassword(username, password)
{
  if( username === 'admin' && password === 'strongPassword098765' )
  {
    return true;
  }
  else
  {
    return false;
  }
}
```

which hint us to make use of this "right" credentials. After we made use of them, admin and strongPassword098765, we were able to find the information that we needed, the flag of interest, after login in the webpage:

<img src="src/4.png" width="700"/>

#### Which OWASP Top 10 is Being Exploited?

The primary OWASP Top 10 vulnerability exploited in this scenario is A7: Identification and Authentication Failures. The application improperly handled authentication by relying on client-side logic (within the secure.js file), where the login credentials (username admin and password strongPassword098765) were hardcoded. This made the credentials easily accessible to anyone who inspected the page’s source code using browser developer tools. The authentication process should have been securely handled on the server side, where sensitive information like usernames and passwords are protected and never exposed to the client.

By storing the credentials directly in the JavaScript file, the application inadvertently exposed sensitive information to the client, making it possible for unauthorized users to bypass the login process altogether. In this case, attackers could simply retrieve the credentials from the client-side code, log in as the admin user, and gain access to sensitive data, such as the flag or other confidential information. 

#### How Can It Be Patched?

To patch the Identification and Authentication Failure vulnerability, the first step is to move authentication logic entirely to the server side. The application should never rely on client-side code (such as JavaScript) for checking or storing credentials. Instead, sensitive data like usernames and passwords should be securely stored on the server, with passwords hashed using a strong hashing algorithm like bcrypt. Authentication should occur via secure, server-side validation, and once authenticated, the server should issue a token or session identifier to manage the user’s session securely. It is useful to also implement multi-factor authentication to secure access, and always make use of HTTPS to encrypt data in transit, protecting credentials from interception. 

### Exercise 3

#### Solution

As in the previous exercise, we are redirected to a webpage, precisely to its login page. After using a generic username and password, we ended up seeing something like this:


<img src="src/5.png" width="700"/>

which, also guided by the hint given (SQLiLite), suggests to us that the SQL injection vulnerabilite can be exploided. 

After checking the resulting sql command, we noticed that, by altering the text given in the password field, we can end up getting advantage of a new sql command. For example, by inserting 
```' OR 1=1 --```, we end up having the following SQL command: ```SELECT id FROM users WHERE password = '' OR 1=1 --```, giving us as a result the following page full of information that is not supposed to be able to the common user:

<img src="src/6.png" width="700"/>

Once in this new page, instead of having a password field, we end up having a "City" field that can be filled with any word and the webpage is supposed to go ahead an try to find the input whereve in the table shown. Also, if checking the contents of this shown table, we ended up readind the sentence "Maybe all the tables". By taking advantage of SQL injection once again, and inserting ```' UNION SELECT null, tbl_name, null FROM sqlite_master WHERE type='table'--``` inside the "City" search field, we end up with the following:

<img src="src/7.png" width="700"/>

showing us all the tables available in the backend of our webpage

<img src="src/8.png" width="700"/>

still using the sql injection vulnerabilitie inside the "City" search field, we can use the following command ```' UNION SELECT null, sql, null FROM sqlite_master WHERE tbl_name='hints'--``` in order to check whats inside the ```hints``` table:

<img src="src/9.png" width="700"/>

with this information, we can use the following command ```' UNION SELECT null, info, null FROM hints--``` to see whats inside the ```info``` field inside the ```hints``` table (still using the "City" search field):

<img src="src/10.png" width="700"/>

getting the following:

<img src="src/11.png" width="700"/>

since this information seems to be useless, we have no choice to go ahead an try the next table after ```hints```

<img src="src/8.png" width="700"/>

which is ```more_table```. For this, we use the command ```' UNION SELECT null, sql, null FROM sqlite_master WHERE tbl_name='more_table'--```, still inside the "City" search field, getting the following:

<img src="src/12.png" width="700"/>

since the ```flag``` field seems to be our fina goal, we went ahead and used the command ```' UNION SELECT null, flag, null FROM more_table--``` and got:

<img src="src/13.png" width="700"/>

our flag of interest: picoCTF{G3tting_5QL_1nJ3c7I0N_l1k3_y0u_sh0ulD_c8ee9477}.

#### Which OWASP Top 10 is Being Exploited?
The vulnerability exploited in this scenario is A1: Injection, specifically SQL Injection (SQLi). SQL Injection occurs when an application allows untrusted data to be directly included in a SQL query, enabling an attacker to manipulate the query and potentially access, modify, or delete data they should not have access to. In this case, by injecting malicious SQL commands into the password and city search fields, you were able to bypass authentication and extract sensitive information, such as database table names and the contents of specific tables, including the flag.

#### How Can It Be Patched?
To patch this SQL Injection (SQLi) vulnerability, you should implement parameterized queries (also known as prepared statements) to ensure that user inputs are treated as data and not executable code. This can be done using proper database libraries or frameworks that support safe query construction. Additionally, input validation should be enforced to reject any suspicious characters or patterns that could be used for SQL injection, such as --, ', or OR 1=1. It's also critical to restrict database privileges to limit what each user or process can access, and error handling should be implemented to prevent detailed database error messages from being shown to users. Finally, using a Web Application Firewall (WAF) can provide an additional layer of defense by detecting and blocking SQL injection attempts.


### Part 2: WAF

We are working on deploying a vulnerable web application (DVWA) and securing it with a Web Application Firewall (WAF). The goal is to show that, after deploying the WAF, common web application attacks (such as SQL injection, File inclusion and so on) are blocked effectively by the firewall. For starters, we first made the following steps:

### **1. Setting Up an EC2 Instance:**

The first step was to create a virtual machine (EC2 instance) on AWS (Amazon Web Services), which provides a cloud environment to host and configure the necessary services. The EC2 instance acts as the server where I will deploy the web application and configure security measures like the Web Application Firewall.

So, we connected to the EC2 instance via **SSH** using the following command:

```bash
ssh -i "Ciberseguridad.pem" ec2-user@ec2-54-91-3-3.compute-1.amazonaws.com
```

This allows us to access the instance's terminal remotely and execute commands on it:

<img src="src/2_1.png" width="500"/>

### **2. Installing and Configuring Nginx Web Server:**

Then, we installed **Nginx** on the EC2 instance to act as the web server that will host the vulnerable web application (DVWA). Nginx is an open-source web server that can handle incoming HTTP requests and serve web content efficiently.

To verify the installation of Nginx, we ran the following command:

```bash
nginx -v

```

The output showed that **Nginx version 1.26.2** is installed:

<img src="src/2_3.png" width="500"/>

We also checked the status of the Nginx service to ensure it is running:

```bash
sudo systemctl status nginx

```

The output confirmed that **Nginx is running** and serving content correctly on the server:

<img src="src/2_3.png" width="500"/>

### **3. Installing and Configuring ModSecurity (WAF):**

Next, we installed **ModSecurity**, an open-source Web Application Firewall (WAF), to protect the DVWA application from common web attacks. ModSecurity helps block attacks like SQL injection, Cross-Site Scripting (XSS), and other malicious behaviors.

To verify the installation of ModSecurity, I ran:

```bash
rpm -qa | grep mod_security

```

<img src="src/2_2.png" width="500"/>

This confirmed that **ModSecurity** (version 2.9.7) and **OWASP ModSecurity Core Rule Set (CRS)** (version 4.2.0) were installed.

We also confirmed that ModSecurity is integrated into Nginx by checking the Nginx configuration:

```bash
sudo nginx -T | grep modsec

```

The configuration output showed that ModSecurity is enabled in Nginx with the following directives:

<img src="src/2_2.png" width="500"/>

- **modsecurity on**: Enables ModSecurity.
- **modsecurity_rules_file**: Points to the configuration file (`/etc/nginx/modsec_includes.conf`) that contains the security rules ModSecurity will apply to incoming HTTP requests.


### **5. Preparing for the Next Steps:**

With **Nginx** running and **ModSecurity** configured, we can proceed with deploying the vulnerable web application (**DVWA**) on the server. DVWA is a deliberately insecure web application that allows for the simulation of attacks, which we can use to test whether ModSecurity is effectively blocking these attacks.

Since we already configured DVWA in a previous homework, we can now just attempt various attacks to test the WAF’s functionality. We will also monitor the **ModSecurity logs** and **Nginx access logs** to verify that the attacks are blocked and recorded.


### **Explanation of the SQL Injection Attempt, ModSecurity Detection, and Nginx Logs**

In this assignment, we conducted a **SQL injection attack** on DVWA deployed on an EC2 instance. The purpose was to test the effectiveness of a **Web Application Firewall (WAF)**, specifically ModSecurity, in blocking common attacks like SQL injection. Additionally, we examined the **Nginx logs** to verify how the server recorded and responded to the attack.

### **SQL Injection Attack**

<img src="src/2_4.png" width="500"/>

We started by attempting a **SQL injection attack** on the `/vulnerabilities/sqli/` endpoint of the DVWA application. The malicious payload used was:

```graphql
?id=a' UNION SELECT "text1","text2";-- -&Submit=Submit

```

<img src="src/2_5.png" width="500"/>

as suggested from the help tab of the DVWA webpage. This payload attempts to exploit a vulnerability in the application by injecting SQL code into the query. Specifically, it tries to combine data from multiple tables (using `UNION SELECT`) to retrieve unauthorized information.

<img src="src/2_6.png" width="500"/>

### **ModSecurity Detection and Blocking**

ModSecurity, a Web Application Firewall (WAF) integrated with **Nginx**, intercepted and blocked the SQL injection attempt. ModSecurity uses a set of predefined rules (part of the **OWASP Core Rule Set - CRS**) to identify malicious behavior, including common attack patterns like SQL injection. Hence, if the WAF was correctly deployed, the attack was detected by multiple ModSecurity rules, which are designed to catch specific types of vulnerabilities.

<img src="src/2_12.png" width="500"/>

1. **Anomaly Scoring**: ModSecurity uses an **anomaly score** system to evaluate requests. In this case, the attack had a score of `13`, which exceeded the threshold of `5`. This triggered the firewall to block the request.
2. **SQL Injection Detection**: ModSecurity detected the malicious SQL injection payload in the request, including the use of keywords like `UNION SELECT`. It matched this pattern to the rules defined in the **OWASP CRS**.

<img src="src/2_7.png" width="500"/>

3. **Blocked Request**: The result was a **403 Forbidden** HTTP response, meaning that ModSecurity blocked the malicious request.
4. **Log Details**: The ModSecurity audit log contains detailed information about the blocked request, including:
    - **Matched rules** (such as those in `REQUEST-942-APPLICATION-ATTACK-SQLI.conf`).
    - **Anomaly score** (`13`), which exceeded the threshold for blocking (`5`).
    - The **403 Forbidden** status code, indicating the attack was blocked.

### Accessing and Analyzing Nginx Logs**

1. **Log File Location**: The Nginx error logs are typically stored in the `/var/log/nginx/` directory. Once the log file is located, we can use the `tail` command to monitor it in real-time. The `f` flag allows us to follow the logs as they are written:
    
```bash
sudo tail -f /var/log/nginx/error.log
    
```
    
This command continuously outputs the latest entries in the log file, allowing you to observe real-time events like blocked requests or server errors.

The SQL injection attack was successfully detected and blocked by the **ModSecurity WAF**. ModSecurity’s anomaly scoring system evaluated the attack as malicious, and the firewall prevented the request from reaching the vulnerable application. The **403 Forbidden** response returned by the server and the **ModSecurity logs** clearly show that the WAF functioned as intended.

By reviewing the **Nginx error logs**, we were able to observe the exact details of the blocked request, including the attacker’s IP address, the request URL, and the specific rules that triggered the block.

### **Exploit Attempt: File Inclusion Vulnerability (LFI)**

<img src="src/2_8.png" width="500"/>

In this phase of the assignment, we attempted to exploit a **File Inclusion vulnerability** in the **DVWA** application. This vulnerability allows an attacker to include files on the server, which could potentially be used to read sensitive system files or execute arbitrary code. The target URL for this test was:

```ruby
http://54.91.3.3/vulnerabilities/fi/?page=include.php

```

We modified the `page` parameter to attempt to include a sensitive system file (`/etc/passwd`), which contains user account information on Unix-like systems. The modified URL was:

```ruby
http://54.91.3.3/vulnerabilities/fi/?page=../../../../../../etc/passwd

```

as suggested by the help tab.

<img src="src/2_10.png" width="500"/>

This URL attempts to use **directory traversal** (`../../../../../../`) to navigate outside the web directory and access the `/etc/passwd` file. If successful, this could leak sensitive information about the system's user accounts.

### **ModSecurity Detection and Blocking**

As soon as the malicious request was detected by **ModSecurity**, the Web Application Firewall (WAF) triggered a block based on its anomaly detection system. The **OWASP Core Rule Set (CRS)**, which ModSecurity uses, includes rules to detect common attack patterns, including **File Inclusion** vulnerabilities.

<img src="src/2_11.png" width="500"/>

The server's response was a **403 Forbidden** status, which indicates that the request was blocked by ModSecurity. Here's the relevant log entry from the **Nginx error logs**:

<img src="src/2_13.png" width="500"/>

```less
2024/12/03 07:49:33 [error] 59142#59142: *317 [client 186.71.155.75] ModSecurity: Access denied with code 403 (phase 2). Matched "Operator `Ge' with parameter `5' against variable `TX:BLOCKING_INBOUND_ANOMALY_SCORE' (Value: `33' ) [file "/etc/nginx/coreruleset/rules/REQUEST-949-BLOCKING-EVALUATION.conf"] [line "222"] [id "949110"] [rev ""] [msg "Inbound Anomaly Score Exceeded (Total Score: 33)"] [data ""] [severity "0"] [ver "OWASP_CRS/4.10.0-dev"] [maturity "0"] [accuracy "0"] [tag "anomaly-evaluation"] [tag "OWASP_CRS"] [hostname "172.31.17.158"] [uri "/vulnerabilities/fi/"] [unique_id "173321217317.950993"] [ref ""], client: 186.71.155.75, server: _, request: "GET /vulnerabilities/fi/?page=../../../../../../etc/passwd HTTP/1.1", host: "54.91.3.3"

```

In this log, ModSecurity detected the attack and blocked it based on the **anomaly score** mechanism. Specifically, the total anomaly score exceeded the threshold of `5`, reaching `33`, which triggered the firewall's blocking mechanism.

### **ModSecurity in action**

The Nginx error log contains useful information about the attack and its blocking:

1. **Client IP**: The IP address of the attacker (`186.71.155.75`).
2. **Request Details**: The request method (`GET`), the malicious URL (`/vulnerabilities/fi/?page=../../../../../../etc/passwd`), and the HTTP version (`HTTP/1.1`).
3. **Anomaly Score**: The total anomaly score is **33**, which exceeded the blocking threshold of `5`.
4. **Blocked Status**: The response to the attacker was a **403 Forbidden** status, indicating that ModSecurity successfully blocked the request.
5. **Matched Rule**: The log shows the specific rule that was triggered (`REQUEST-949-BLOCKING-EVALUATION.conf`), which is responsible for evaluating and blocking anomalous requests.

### **What Could Have Happened Without ModSecurity?**

Without ModSecurity or a similar WAF, the directory traversal attack could have successfully included the `/etc/passwd` file, potentially exposing sensitive user information such as usernames and hashed passwords. In a real-world scenario, an attacker could exploit this to escalate privileges or gather more information for further attacks.

### **Exploit Attempt: Command Injection**

<img src="src/2_14.png" width="500"/>

For this phase of the assignment, we tested the **Command Injection** vulnerability in the DVWA application. Command injection vulnerabilities allow an attacker to execute arbitrary commands on the server, potentially compromising the system.

In this case, the vulnerable input is an IP address field where users are asked to enter an IP address. To test for command injection, we submitted the following payload:

```bash
127.0.0.1 && dir

```

as suggested by the help tab:

<img src="src/2_15.png" width="500"/>

Here, `127.0.0.1` is a valid IP address (representing the local loopback address), but the `&& dir` part of the input is a command separator that attempts to execute the `dir` command (which lists directories in Windows). If the system is vulnerable, it would attempt to run the `dir` command alongside the valid IP address, revealing the directory structure of the server.


### **ModSecurity Detection and Blocking**

<img src="src/2_16.png" width="500"/>

<img src="src/2_17.png" width="500"/>

As with the previous tests, **ModSecurity** — a Web Application Firewall (WAF) — detected the malicious request and blocked it. ModSecurity is configured with rules from the **OWASP Core Rule Set (CRS)** to detect various types of attacks, including **Command Injection**.

Upon submitting the malicious input, ModSecurity evaluated the request and found it to be anomalous. It assigned an **anomaly score** and determined that the request should be blocked.

The **Nginx error log** contains the following entry that shows how the attack was detected and blocked:

<img src="src/2_18.png" width="500"/>

```less
2024/12/03 07:53:21 [error] 59142#59142: *323 [client 186.71.155.75] ModSecurity: Access denied with code 403 (phase 2). Matched "Operator `Ge' with parameter `5' against variable `TX:BLOCKING_INBOUND_ANOMALY_SCORE' (Value: `8' ) [file "/etc/nginx/coreruleset/rules/REQUEST-949-BLOCKING-EVALUATION.conf"] [line "222"] [id "949110"] [rev ""] [msg "Inbound Anomaly Score Exceeded (Total Score: 8)"] [data ""] [severity "0"] [ver "OWASP_CRS/4.10.0-dev"] [maturity "0"] [accuracy "0"] [tag "anomaly-evaluation"] [tag "OWASP_CRS"] [hostname "172.31.17.158"] [uri "/vulnerabilities/exec/"] [unique_id "173321240142.447436"] [ref ""], client: 186.71.155.75, server: _, request: "POST /vulnerabilities/exec/ HTTP/1.1", host: "54.91.3.3", referrer: "http://54.91.3.3/vulnerabilities/exec/"

```

### **Breaking Down the Log Entry**

- **Client IP**: The attacker’s IP address is `186.71.155.75`.
- **Request Details**: The HTTP request method is `POST`, and the URL being accessed is `/vulnerabilities/exec/`. The attack attempt was made with the malicious input (`127.0.0.1 && dir`).
- **Anomaly Score**: ModSecurity assigned an **anomaly score of 8**, which exceeded the threshold of `5` (as configured in the ModSecurity rules). This score was enough to trigger the blocking mechanism.
- **Blocking Action**: The request was blocked, and the server returned a **403 Forbidden** response to the attacker, indicating that the request was denied.

The request was blocked because it triggered a ModSecurity rule, specifically one that is part of the **OWASP CRS**. The rule responsible for blocking the attack is in the file `REQUEST-949-BLOCKING-EVALUATION.conf`. This rule is designed to evaluate the **anomaly score** of incoming requests and block them if the score exceeds a predefined threshold.

### **What Could Have Happened Without ModSecurity?**

Without ModSecurity, if the **Command Injection** vulnerability had not been protected by a WAF, the command `127.0.0.1 && dir` could have executed on the server. This would have caused the `dir` command to run, potentially revealing sensitive file and directory information about the system.

For example, if the server is running a Windows OS, the `dir` command would list directories and files in the current directory. If the server were running Linux, it could execute any available shell commands, potentially exposing sensitive data or allowing further exploitation.